# Рекомендация тарифов

В моем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — я проводил её в предыдущем проекте.

Нужно построить модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. 

<div>
    
<b> Описание проекта</b>
    
В данном проекте необходимо построить модель, которая будет подбирать подходящий тариф для пользователя
    
Данный проект состоит из пяти основных частей:
    
* Открытие файла и изучение данных
* Разделение исходных данных на обучающую, валидационную и тестовую выборки
* Исследование качества разных моделей, с разными значениями гиперпараметров
* Проверка качества модели на тестовой выборке
* Проверка модели на вменяемость
* Общий вывод
    
</div>

## Открытие файла и изучение данных

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from random import randint
import warnings
warnings.filterwarnings('ignore')
from sklearn.dummy import DummyClassifier

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Из таблицы видно, что в столбце "is_ultra" показано, является ли эта строка тарифом Ультра ("is_ultra" = 1) или тарифом Смарт ("is_ultra" = 0)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


## Разделение исходных данных на обучающую, валидационную и тестовую выборки

In [ ]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

In [ ]:
train, valid, test = np.split(df.sample(frac=1, random_state=42),[int(.6*len(df)), int(.8*len(df))])

In [ ]:
features_train = train.drop(['is_ultra'], axis=1)
target_train = train['is_ultra']

In [ ]:
features_valid = valid.drop(['is_ultra'], axis=1)
target_valid = valid['is_ultra']

In [ ]:
features_test = test.drop(['is_ultra'], axis=1)
target_test = test['is_ultra']

In [ ]:
print('Размер таблицы признаков для тренировочной выборки:', len(features_train))
print('Размер таблицы целевых значений для тренировочной выборки:', len(target_train))

Размер таблицы признаков для тренировочной выборки: 1928
Размер таблицы целевых значений для тренировочной выборки: 1928


In [ ]:
print('Размер таблицы признаков для валидационной выборки:', len(features_valid))
print('Размер таблицы целевых значений для валидационной выборки:', len(target_valid))

Размер таблицы признаков для валидационной выборки: 643
Размер таблицы целевых значений для валидационной выборки: 643


In [ ]:
print('Размер таблицы признаков для тестовой выборки:', len(features_test))
print('Размер таблицы целевых значений для тестовой выборки:', len(target_test))

Размер таблицы признаков для тестовой выборки: 643
Размер таблицы целевых значений для тестовой выборки: 643


## Исследование качества разных моделей, с разными значениями гиперпараметров

    
Исследую модели в следующем порядке:
    
* логистическая регрессия
* случайный лес
* дерево решений
 

<div>
    
<b> Логистическая регрессия</b>
    
</div>

In [ ]:
model = LogisticRegression(random_state=12345)
model.fit(features_train, target_train)
result = model.score(features_valid, target_valid)
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7309486780715396


<div>
    
<b> Случайный лес</b>
    
</div>

In [ ]:
best_model = None
best_result = 0
for est in range(1, 10):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result:
        best_model_1 = model
        best_result = model.score(features_valid, target_valid)
        est1 = est

print("Accuracy наилучшей модели на валидационной выборке:", best_result)
print("Количество деревьев в модели:", est1)

Accuracy наилучшей модели на валидационной выборке: 0.7884914463452566
Количество деревьев в модели: 4


<div>
    
<b> Дерево решений</b>
    
</div>

In [ ]:
best_model = None
best_result = 0
for depth in range(1, 10):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(features_train, target_train)
    predictions = model.predict(features_valid)
    result = accuracy_score(predictions, target_valid)
    if result > best_result:
        best_model_2 = model
        best_result = result
        depth1 = depth
        
print("Accuracy лучшей модели:", best_result)
print("Глубина модели:",  depth1)

Accuracy лучшей модели: 0.7900466562986003
Глубина модели: 9


<div>
    
<b> Вывод по разделу:</b>
    
Исходя из значений accuracy лучше всего предсказывает Дерево решений. Но необходимо проверить, не переобучилась ли модель.   
    
</div>

## Проверка качества модели на тестовой выборке

In [ ]:
predictions = best_model_2.predict(features_test)
result = accuracy_score(predictions, target_test)
        
print("Accuracy модели:", result)

Accuracy модели: 0.8009331259720062


Поскольку оценка качества модели на тестовых данных почти равна оценке на валидационных, можно сказать, что модель не переобучилась.

## Проверка модели на вменяемость

In [ ]:
my_list = [randint(0, 1) for i in range(len(predictions))]

In [ ]:
result = accuracy_score(my_list, target_test)
print("Accuracy рандома:", result)

Accuracy рандома: 0.49922239502332816


У случайно выбранных чисел accuracy составляет 0.5, а у итоговой модели 0.8, значит выбранная модель адекватна.

In [ ]:
dum_model = DummyClassifier(strategy='constant', constant=target_train[0])
dum_model.fit(features_train, target_train)
predictions_1 = dum_model.predict(features_test)
result = accuracy_score(predictions_1, target_test)
print("Accuracy of DummyClassifier:", result)

Accuracy of DummyClassifier: 0.6905132192846034


Запустил модель DummyClassifier. Значение acceracy получилось 0,69. Это доля нулей в датафрейме target_test. Поскольку в итоговой модели для предсказаний тарифа значение acceracy больше значения для DummyClassifier, значит модель адекватна.

## Общий вывод

В данном проекте было произведено сравнение нескольких моделей машинного обучения.

Предварительно данные были разбиты на тренировочную, валидационную и тестовую выборки. Затем были применены 3 вида обучающих моделей: логистическая регрессия, случайный лес, дерево решений. После настройки и примения моделей к тренировочной выборке была произведена оценка каждой модели. Лучшее качество модели показало Дерево решений. Случайный лес показал почти такие же результаты (отставание от дерева решений минимально). Логистическая регрессия показала оценку 0.73, что не подходит под условия задачи.

После того, как модель с оптимальными гиперпараметрами была определена, была проведена проверка на переобучение, которая показала, что модель не переобучена. 

Далее была проведена проверка на вменяемость: был составлен список со случайными числами (0 или 1), той же длины, что и список предсказанный моделью. Затем было произведено сравнение списка случайных чисел со списком правильных ответов. Соответствие случайных и правильных значений было в половине случаев. Если посмотреть на значение accuracy модели Дерева решений (0.8) то можно сказать, что полученная модель адекватна.